# Resolution in Python

Brainstorming:
- Descriptor
    * Rollback example
    * Proxy pattern (lazy)
- MRO: just mention, topic of another lecture
- `getattr`, `setattr`
- Monkey Patching and mocking
- Mention slot?

## Basic lookup

When accessing the attribute of an object, Python will proceed as follow:
1. invoke que the `__getattribute__` method, which is responsible for
2. looking into the instance `__dict__` for the attribute
3. looking into the class `__dict__` if the former fails
4. delegate to the `__getattr__` method for custom handling
5. if `__getattr__` is not implemented, an `AttributeError` is raised.

This lookup will be refined as we introduced new concepts.

> :warning: :skull: Note that if the class is defined with `slot`, the instance `__dict__` will be missing (unless it is explictly re-created as a slot).

Let's invsetigate the lookup step by step.

In [ ]:
# Attribute lookup
class Rectangle:

    def __init__(self, width, height):
        self.width = width
        self.height = height


    def __getattribute__(self, name):
        print(f"Accessing attribute: {name}")
        return super().__getattribute__(name)



r = Rectangle(10, 20)
print(r.width)
print(r.__dict__)

print("---")
r.width = 15
print(r.__dict__)

> :warning: :skull: Note that Python is not using the `__getitem__` method of the dictionary (at least in CPython) but does a more direct lookup

In the case of a class attribute, or method, we end up in the class `__dict__`

In [ ]:
# Method and class attribute lookup
class Rectangle:

    _AUTHOR_ = "JMB"

    def __init__(self, width, height):
        self.width = width
        self.height = height


    def __getattribute__(self, name):
        print(f"Accessing attribute: {name}")
        return super().__getattribute__(name)

    def area(self):
        return self.width * self.height


r = Rectangle(10, 20)
print(r.__dict__)  # Only attributes

print("---")
r._AUTHOR_ # goes through the __getattribute__ method
r.area()   # goes through the __getattribute__ method

print("---")
print(Rectangle.__dict__)  # Contains _AUTHOR_ and area

Interestingly, when performing a lookup for a method, Python finds it in the **class** `__dict__` and then binds it to the instance. This is different from the following

In [ ]:
class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        def area():
            return self.width * self.height
        self.area = area

r = Rectangle(10, 20)
print(r.__dict__)  # Contains area as well


## Inheritance

### Simple inheritance

What happens in case of inheritance?

In [43]:
class Rectangle:

    def __init__(self, width, height):
        self.width = width
        self.height = height

    def area(self):
        return self.width * self.height
    

class Square(Rectangle):
    def __init__(self, side):
        super().__init__(side, side)
    

print(Square.__dict__)  # area not in Square.__dict__
print(Square.area)  # Accessible even though it is not in Square.__dict__

{'__module__': '__main__', '__init__': <function Square.__init__ at 0x7f5d64250a60>, '__doc__': None}
<function Rectangle.area at 0x7f5d64250430>


:question: Consider the following piece of code. Explain in which `__dict__` will the following be found (if any)
- `__o.a_value`
- `__o.b_value`
- `__o.a_method_1`
- `__o.a_method_2`
- `__o.b_method`
- `__o.CONSTANT`
- `__o.CONSTANT_2`
- `__o.c_method`

In [ ]:
class A:
    CONSTANT = 3.14
    CONSTANT_2 = 42

    def __init__(self, a_value):
        self.a_value = a_value

    def a_method_1(self):
        pass

    def a_method_2(self):
        pass

class B(A):
    CONSTANT = 6371

    def __init__(self, a_value, b_value):
        super().__init__(a_value)
        self.b_value = b_value

    def b_method(self):
        pass

    def a_method_1(self):
        pass

__o = B(10, 750)
__o.c_method = lambda: -1

-1

### Multiple inheritance and MRO

> How does the lookup proceed in case of complex inheritance? For instance
> - Diamond problem
> - MRO
> - Mention limit with `__init__` as warning, but refer to the next training

Python allows for Multiple Inheritance. An issue that arises from such mechanism is called the Diamond problem. Consider the following inheritance diagram:
```
      Person        
     /      \       
Teacher  Researcher 
     \      /       
    Professor       
```
What happens if a method of a `Professor` instance is called when
- the method is defined only in `Professor`;
- the method is defined only in `Person`;
- the method is re-defined both in `Teacher` and `Researcher`?

The most common such method is the `__init__` dunder.

In [30]:
class Person:
    def __init__(self):
        print("Person init")

class Teacher(Person):
    def __init__(self):
        print("Teacher init")
        super().__init__()

class Researcher(Person):
    def __init__(self):
        print("Researcher init")
        super().__init__()


In [41]:
class Professor1(Researcher, Teacher):
    def __init__(self):
        print("Professor init (R, T)")
        super().__init__()

_ = Professor1()

Professor init (R, T)
Researcher init
Teacher init
Person init


In [42]:
class Professor2(Teacher, Researcher):
    def __init__(self):
        print("Professor init (T, R)")
        super().__init__()

_ = Professor2()

Professor init (T, R)
Teacher init
Researcher init
Person init


Python uses the C3 linearization algorithm to determine the orders of the calls. In practice, a deterministic order is established and stored in the `__mro__` (Multi-inheritance Resolution Order) field of the **class**, also accessible via the `mro` class method:

In [40]:
print("Professor 1:", Professor1.__mro__, Professor1.mro())
print("Professor 2:", Professor2.__mro__)

Professor 1: (<class '__main__.Professor1'>, <class '__main__.Researcher'>, <class '__main__.Teacher'>, <class '__main__.Person'>, <class 'object'>) [<class '__main__.Professor1'>, <class '__main__.Researcher'>, <class '__main__.Teacher'>, <class '__main__.Person'>, <class 'object'>]
Professor 2: (<class '__main__.Professor2'>, <class '__main__.Teacher'>, <class '__main__.Researcher'>, <class '__main__.Person'>, <class 'object'>)


the `super` call follows the MRO. See the "Advanced Inheritance" chapter for more on multiple inheritance.

> Note how the last one if always the `object` base class.

> :warning: The C3 algorithm is neither a breadth-first, nor depth-first exploration of the hierarchy DAG. Rather it is a merging algorithm which aims to enforce three poperties (local precedence, monotonicity and consistency).

Even in the case of simple inheritance, Python relies on the MRO for the lookup, although it is trivial then:

In [59]:
Square.mro()

[__main__.Square, __main__.Rectangle, object]

:wrench: Exercise: determine the output of `G().greet()`. You can use the MRO

In [60]:
class A:
    def greet(self):
        return "Hello from A"
    
class B(A):
    def greet(self):
        return "Hello from B"

class C(A):
    def greet(self):
        return "Hello from C (property)"

class D(B, C):
    def greet(self):
        return super().greet() + " @ D"

class E:
    def greet(self):
        return "Hello from E"
    
class F(E, C):
    pass

class G(D, F):
    pass

# G.mro()

## Fallback `__getattr__`

The last step of the resolution in case of miss is to invoke the `__getattr__` method, which by default raises a `AttributeError` exception. 

In [ ]:
class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def area(self):
        return self.width * self.height

    def __getattr__(self, name):
        print(f"__getattr__ called for: {name}")
        if name == "perimeter":
            return lambda: 2*self.width + 2*self.height  # Note that self is not passed
        raise AttributeError(f"{name} not found (__getattr__)")
    
r = Rectangle(10, 20)
print("Width:", r.width)
print("---")
print("Area:", r.area())
print("---")
print("Perimeter:", r.perimeter())
print("---")
print(r.radius)

Width: 10
---
Area: 200
---
__getattr__ called for: perimeter
Perimeter: 60
---
__getattr__ called for: radius


AttributeError: radius not found (__getattr__)

Python exposes the `getattr` method to call the `__getattr__` dunder, with the nice addition of returning a default value:

In [63]:
getattr??

Docstring:
getattr(object, name[, default]) -> value

Get a named attribute from an object; getattr(x, 'y') is equivalent to x.y.
When a default argument is given, it is returned when the attribute doesn't
exist; without it, an exception is raised in that case.
Type:      builtin_function_or_method

:wrench: Write a `LogCallDecorator` class, which prints every call made to the decorated object but otherwise behaves exactly as the decorated object.

In [ ]:
from typing import Any


class LogCallDecorator:
    def __init__(self, __o: Any):
        self.__o = __o
    
    # TODO 

There are several cases where overriding the `__getattr__` method is handy, mostly in advance cases. This include mocking, lazyness, and design patterns (eg. decorator, builder, vistitor).

## Descriptor

### The notion of descriptor

It is time to revisit the lookup thanks to the notion of descriptors:
1. invoke que the `__getattribute__` method, which is responsible for
2. looking into the instance `__dict__` for the attribute
3. looking into the class `__dict__` if the former fails
4. **Apply descriptor if something is found** else
5. delegate to the `__getattr__` method for custom handling
6. if `__getattr__` is not implemented, an `AttributeError` is raised.

Descriptors let objects customize attribute lookup, storage, and deletion (https://docs.python.org/3/howto/descriptor.html). This is a powerful mechanism. Technically, an descriptor is any object which defines `__get__`, `__set__`, or `__delete__` method.

> :warning: the descriptor is only applied as a class attribute (not instance, nor in `__getattr__`). TODO

In [ ]:
import datetime as dt
from time import sleep

class Now:
    def __get__(self, instance, owner):
        print("__get__ called")
        return dt.datetime.now()
    
class MyClass:
    now = Now()


obj = MyClass()
print(obj.now) 
sleep(10)
print(obj.now)  

__get__ called
2025-09-01 13:38:26.674341
__get__ called
2025-09-01 13:38:36.684837


It is often useful for a descriptor to know the name it is associated with. Python allows to do that via the `__

In [ ]:
class Tracker:
    def __init__(self, value):
        self.value = value
        self.name = None

    def __set_name__(self, owner, name):
        # print(f"Setting name: {name}")
        self.name = name

    def __get__(self, instance, owner):
        print(f"accessing {self.name if self.name else 'unknown'}")
        return self.value
    
class MyClass:
    x = Tracker(10)
    y = Tracker(20)
    # z = x  # overrides name


obj = MyClass()
print(obj.x)
print(obj.y)
print(obj.x)

accessing z
10
accessing y
20
accessing z
10


### Properties

In the example above, notice how we are just accessing, not calling. This behavior should be reminiscent of `property`. We can re-implement the behavior of a property with descriptor and function decorator.

In [80]:
def property_(f):
    class _Property:
        def __init__(self, func):
            self.func = func
            self.__doc__ = getattr(func, "__doc__", None)

        def __get__(self, instance, owner):
            if instance is None:
                return self  # like built-in property when accessed on the class
            if self.func is None:
                raise AttributeError(f"unreadable attribute")
            return self.func(instance)

    return _Property(f)


class Circle:
    def __init__(self, radius):
        self.radius = radius

    @property_
    def pi(self):
        return 3.14

    def area(self):
        return self.pi * self.radius ** 2



c = Circle(10)
print(c.pi)

3.14


:wrench: Using the `__set_name__` method, create a tracker descriptor which prints the name of the variable each time it is accessed

accessing x
10
accessing y
20
accessing x
10



- Revisit the lookup resolution
- Illustrate with rollback (and lazy?)


## Stateful descriptor

In [ ]:
# Quid if getattr returns a descriptor?

## Monckey Patching

Discuss `setattr` here

## Misc
- Slots
- `dir`  https://docs.python.org/3/library/functions.html#dir
- `hasattr`

## Quizz

- When should you re-implement `__getattribute__`?


TODO exos